Verifying all libraries

In [1]:
# Check if all required libraries are installed
required_libraries = ["os", "time", "pandas", "subprocess", "Bio"]
missing_libraries = []

for lib in required_libraries:
    try:
        __import__(lib)
    except ImportError:
        missing_libraries.append(lib)

if missing_libraries:
    print("The following required libraries are missing:")
    for lib in missing_libraries:
        print(f" - {lib}")
    print("\nPlease install them using pip. Example:")
    print("pip install pandas biopython")
    exit()  # Exit the script if libraries are missing

# Proceed with imports
import os
import time
import pandas as pd
import subprocess
from Bio.Blast import NCBIWWW
from Bio import Entrez

# Verify BLAST is installed
try:
    subprocess.run(["blastp", "-version"], capture_output=True, text=True, check=True)
    print("BLAST is installed and available.")
except FileNotFoundError:
    print("BLAST is not installed or not available in the PATH. Please install it before proceeding.")
    exit()
    
# Initial message if everything is fine
print("All required libraries are installed, and BLAST is available.")


BLAST is installed and available.
All required libraries are installed, and BLAST is available.


Check if Protein names are valid

In [2]:
from Bio import Entrez
import pandas as pd

# Load Excel file and extract gene names and species
df_path = "Gene_data.xlsx"
df = pd.read_excel(df_path, sheet_name='Gene_names_ids', header=0)

# Drop rows with any NaN values
df = df.dropna()

# Set email for NCBI
Entrez.email = "youremail@example.com"

# Function to print details for genes with no protein sequences found
def print_missing_protein_sequences(df):
    for index, row in df.iterrows():
        gene_ID = row['Gene ID']
        species = row['Species']  # Assume 'Species' is the column name for species information

        try:
            # Search for the protein related to the gene in the specified species
            query = f"{gene_ID}[Gene] AND {species}[Organism]"
            search_handle = Entrez.esearch(db="protein", term=query)
            search_record = Entrez.read(search_handle)
            search_handle.close()

            # Print gene-species pairs where no protein sequences are found
            if not search_record["IdList"]:
                # Print the full row details for missing protein sequences
                print(f"Missing protein sequence for: {row.to_dict()}")
        except Exception as e:
            print(f"Error processing {gene_ID} in {species}: {str(e)}")

# Run the function to print missing proteins
print_missing_protein_sequences(df)


Missing protein sequence for: {'Gene name': 'Test1234545', 'Gene ID': 'testtestest', 'Category': 'test test. Test', 'Species': 'Drosophila melanogaster'}


Load and Download all genes from NCBI library 

In [6]:
import os  # For directories
from Bio import Entrez  # For accessing the NCBI database
import pandas as pd  # For handling the Excel file
import time  # To add delays between requests

# Load Excel file and extract gene names, species, and IDs
df_path = "Gene_data.xlsx"  # Update with the correct file name if needed
sheet_name = 'Dros_gene_names_full'  # Specify the sheet name

df = pd.read_excel(df_path, sheet_name=0)

# Drop rows with any missing values
df = df.dropna()

# Set email for NCBI
Entrez.email = "youremail@example.com"  # Please enter your email

# Define output directories
output_dir_grouped = "Protein_FASTA_Files_grouped"  # Grouped by gene and species
output_dir_indiv = "Protein_FASTA_Files_indiv"  # Same files as grouped but stored individually
os.makedirs(output_dir_grouped, exist_ok=True)
os.makedirs(output_dir_indiv, exist_ok=True)

# Function to clear the console (works for most environments)
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')

# Function to download and save protein sequences
def download_and_save_sequences(df):
    not_found_proteins = []  # List to track protein sequences that were not found

    for gene_name, group in df.groupby('Gene name'):  # Group by gene name
        # Replace spaces and special characters in gene names
        safe_gene_name = gene_name.replace(" ", "_").replace("/", "_")

        # Create the grouped folder for each gene
        gene_grouped_dir = os.path.join(output_dir_grouped)
        os.makedirs(gene_grouped_dir, exist_ok=True)

        for species, species_group in group.groupby('Species'):  # Group by species within the gene
            # Replace spaces and special characters in species names
            safe_species_name = species.replace(" ", "_").replace("/", "_")

            # Path for the grouped FASTA file (Species_Gene_combined.fasta)
            grouped_fasta_filename = f"{safe_species_name}_{safe_gene_name}_combined.fasta"
            grouped_fasta_path = os.path.join(gene_grouped_dir, grouped_fasta_filename)

            # Path for the individual FASTA file (same as grouped, but in a separate folder)
            indiv_fasta_path = os.path.join(output_dir_indiv, grouped_fasta_filename)

            try:
                with open(grouped_fasta_path, "w") as grouped_fasta_file, open(indiv_fasta_path, "w") as indiv_fasta_file:
                    for index, row in species_group.iterrows():
                        gene_id = row['Gene ID']  # Replace with the column name for Gene ID

                        # Retry mechanism for NCBI requests
                        retries = 3
                        for attempt in range(retries):
                            try:
                                # Search for the protein related to the gene in the specified species
                                search_term = f"{gene_id}[Gene] AND {species}[Organism]"
                                search_handle = Entrez.esearch(db="protein", term=search_term)
                                search_record = Entrez.read(search_handle)
                                search_handle.close()

                                # Check if any protein sequences were found
                                if search_record["IdList"]:
                                    for sequence_id in search_record["IdList"]:
                                        fetch_handle = Entrez.efetch(db="protein", id=sequence_id, rettype="fasta", retmode="text")
                                        sequence_data = fetch_handle.read()
                                        fetch_handle.close()

                                        # Write sequence data to both grouped and individual files
                                        grouped_fasta_file.write(sequence_data)
                                        indiv_fasta_file.write(sequence_data)
                                    break  # Exit retry loop if successful
                                else:
                                    # Log not-found cases
                                    not_found_proteins.append((gene_id, species))
                                    print(f"No protein sequences found for {gene_id} in {species}")
                                    break
                            except Exception as e:
                                if attempt < retries - 1:
                                    print(f"Retrying ({attempt + 1}/{retries}) for {gene_id} in {species}...")
                                    time.sleep(5)  # Wait 5 seconds before retrying
                                else:
                                    print(f"Failed after {retries} attempts for {gene_id} in {species}")
                                    raise e  # Raise the exception after all retries fail
            except Exception as e:
                print(f"Error retrieving data for {species} in gene {gene_name}: {e}")

    # Clear console before printing the summary
    clear_console()

    # Print summary of not found proteins
    print("\nSummary: Proteins Not Found")
    if not_found_proteins:
        for gene_id, species in not_found_proteins:
            print(f"Gene ID: {gene_id}, Species: {species}")
    else:
        print("All proteins were successfully retrieved!")

    print("All genes processed successfully!")

# Run the function
download_and_save_sequences(df)


No protein sequences found for testtestest in Drosophila melanogaster

Summary: Proteins Not Found
Gene ID: testtestest, Species: Drosophila melanogaster
All genes processed successfully!


Create a BLAST database

In [ ]:
import os  # For directory management
import subprocess  # For running external commands

# Define the base folder
base_dir = "Protein_FASTA_Files_grouped"  # Folder containing gene subfolders
db_dir = "blast_databases"  # Root folder to store BLAST databases

# Create the root database folder if it doesn't exist
os.makedirs(db_dir, exist_ok=True)

# Function to clean and validate FASTA files
def clean_fasta(input_file):
    """Cleans and validates a FASTA file by simplifying the header and removing invalid characters."""
    with open(input_file, "r") as infile:
        lines = infile.readlines()

    with open(input_file, "w") as outfile:  # Overwrite the original file
        for line in lines:
            if line.startswith(">"):
                # Retain only the sequence ID from the header
                sequence_id = line.split()[0]  # Keep everything before the first space
                outfile.write(sequence_id + "\n")  # Write the cleaned header
            else:
                # Remove spaces and write the cleaned sequence
                outfile.write(line.strip() + "\n")

# Function to clear the console (works for most environments)
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')

# List to track failed database creations
failed_dbs = []

# Loop through the gene folders to create BLAST databases
for gene_folder in os.listdir(base_dir):
    gene_path = os.path.join(base_dir, gene_folder)  # Path to each gene folder
    if os.path.isdir(gene_path):  # Only process folders
        # Create a separate subfolder in `blast_databases` for each gene
        gene_db_folder = os.path.join(db_dir, gene_folder)
        os.makedirs(gene_db_folder, exist_ok=True)

        # Process each FASTA file within the gene folder
        for fasta_file in os.listdir(gene_path):
            if fasta_file.endswith(".fasta"):
                fasta_path = os.path.join(gene_path, fasta_file)  # Path to the FASTA file

                # Clean the FASTA file (overwrite the original file)
                clean_fasta(fasta_path)

                # Create a BLAST database in the gene-specific folder
                db_name = os.path.join(gene_db_folder, os.path.splitext(fasta_file)[0])
                try:
                    subprocess.run(
                        ["makeblastdb", "-in", fasta_path, "-dbtype", "prot", "-out", db_name],
                        check=True,
                    )
                except subprocess.CalledProcessError as e:
                    failed_dbs.append((fasta_file, str(e)))

# Clear console before displaying results
clear_console()

# Final message with failed databases
if failed_dbs:
    print("\nFailed Database Creations:")
    for fasta_file, error in failed_dbs:
        print(f"File: {fasta_file}, Error: {error}")
else:
    print("All gene folders processed successfully without errors!")


Be able to choose the species to use as a query using input

In [ ]:
import os
import pandas as pd
import subprocess
from io import StringIO

# Paths and parameters
main_gene_folder = "Protein_FASTA_Files_grouped"        # Main folder containing gene subfolders
db_folder = "blast_databases"                  # Root folder for BLAST databases
evalue = 0.001                                 # E-value threshold for BLAST
max_hits = 9999                                # Max hits to retain from BLAST results (we want all)

# Function to sanitize names
def sanitize_name(name):
    """Remove special characters and spaces from a string."""
    return "".join(c for c in name if c.isalnum() or c == "_")

# Function to clear the console (works for most environments)
def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')

# Function to run BLAST with separated self-comparison and all-vs-others
def run_cross_species_blast(main_gene_folder, db_folder, evalue, max_hits):
    failed_blasts = []  # List to track failed BLAST commands
    
    # Prompt the user to enter the input species manually
    input_species = input("Enter the species to use as input for BLAST comparisons, pleas use _ as spaces e.g Drosophila_melanogaster: ").strip()
    
    # Create the root output folder named based on the selected species
    output_root_folder = f"blast_results_{sanitize_name(input_species)}_vs_all"
    os.makedirs(output_root_folder, exist_ok=True)

    # Loop through each gene folder
    for gene_folder in os.listdir(main_gene_folder):
        gene_path = os.path.join(main_gene_folder, gene_folder)
        if os.path.isdir(gene_path):  # Check if it's a directory
            print(f"Processing gene: {gene_folder}")
            
            # Iterate over species within the gene folder
            species_files = [f for f in os.listdir(gene_path) if f.endswith("_combined.fasta")]
            
            if f"{input_species}_combined.fasta" not in species_files:
                print(f"Species '{input_species}' not found in gene folder '{gene_folder}'. Skipping...")
                continue
            
            # Create a dedicated output folder for the current gene
            gene_output_folder = os.path.join(output_root_folder, gene_folder)
            os.makedirs(gene_output_folder, exist_ok=True)
            
            input_species_file = f"{input_species}_combined.fasta"
            query_path = os.path.join(gene_path, input_species_file)

            # Prepare CSV files for self-comparison and species-vs-others
            self_comparison_csv = os.path.join(gene_output_folder, f"{input_species}_self_comparison.csv")
            all_vs_csv = os.path.join(gene_output_folder, f"{input_species}_vs_all.csv")
            self_results_df = pd.DataFrame(columns=[
                "Query ID", "Subject ID", "% Identity", "Alignment Length", "Mismatches", 
                "Gap Opens", "Query Start", "Query End", "Subject Start", "Subject End", 
                "E-value", "Bit Score"
            ])
            all_vs_results_df = pd.DataFrame(columns=[
                "Query ID", "Subject ID", "% Identity", "Alignment Length", "Mismatches", 
                "Gap Opens", "Query Start", "Query End", "Subject Start", "Subject End", 
                "E-value", "Bit Score", "Target Species"
            ])
            
            # BLAST the input species against all others
            for target_file in species_files:
                target_species = target_file.split("_combined")[0]
                
                # Construct the BLAST database name
                target_db_folder = os.path.join(db_folder, gene_folder)
                target_db = os.path.join(target_db_folder, f"{target_species}_combined")
                
                # Check if the BLAST database exists before running
                if not os.path.exists(f"{target_db}.phr") or not os.path.exists(f"{target_db}.pin") or not os.path.exists(f"{target_db}.psq"):
                    failed_blasts.append((input_species, target_species, "Database not found"))
                    continue
                
                # Run BLASTp
                blast_cmd = [
                    "blastp",
                    "-query", query_path,
                    "-db", target_db,
                    "-evalue", str(evalue),
                    "-outfmt", "6",
                    "-max_target_seqs", str(max_hits)
                ]
                try:
                    result = subprocess.run(blast_cmd, capture_output=True, text=True, check=True)
                    # Process BLAST results
                    if result.stdout:
                        blast_results = pd.read_csv(
                            StringIO(result.stdout), 
                            sep="\t", 
                            names=[
                                "Query ID", "Subject ID", "% Identity", "Alignment Length", "Mismatches", 
                                "Gap Opens", "Query Start", "Query End", "Subject Start", "Subject End", 
                                "E-value", "Bit Score"
                            ]
                        )
                        if input_species == target_species:  # Self-comparison
                            self_results_df = pd.concat([self_results_df, blast_results], ignore_index=True)
                        else:  # Other species
                            blast_results["Target Species"] = target_species
                            all_vs_results_df = pd.concat([all_vs_results_df, blast_results], ignore_index=True)
                except subprocess.CalledProcessError as e:
                    failed_blasts.append((input_species, target_species, str(e)))
            
            # Save results to CSV files
            self_results_df.to_csv(self_comparison_csv, index=False)
            all_vs_results_df.to_csv(all_vs_csv, index=False)
    
    # Clear console before printing failures
    clear_console()
    
    # Print any failed BLAST commands
    if failed_blasts:
        print("Failed BLAST Commands:")
        for input_species, target_species, error in failed_blasts:
            print(f"Input Species: {input_species}, Target Species: {target_species}, Error: {error}")
    else:
        print("All BLAST commands completed successfully!")

# Run the function
run_cross_species_blast(main_gene_folder, db_folder, evalue, max_hits)
